In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist


(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(-1, 28, 28, 1) / 255.0
x_test = x_test.reshape(-1, 28, 28, 1) / 255.0

def generate_random_number(number,x_data,y_data):
    num_samples = number
    data_numb=np.zeros((num_samples, 28, 100, 1), dtype = np.float32)
    data_labe=np.empty((num_samples, 1), dtype = np.chararray)
    for i in range(num_samples):
        digit1 = np.random.choice(y_data)
        digit2 = np.random.choice(y_data)
        digit3 = np.random.choice(y_data)
        digit4 = np.random.choice(y_data)
        digit5 = np.random.choice(y_data)
        
        length = np.random.randint(1, 6)
        canvas = np.zeros((28, 100, 1), dtype=np.float32)
        labe = np.zeros(length,dtype=np.int_)
        
        lent_front = 0
        for n in range(length):
            digit_labe =[digit1, digit2, digit3, digit4, digit5]
            digit = digit_labe[n]
            id=np.random.choice(np.where(y_train == digit)[0])
            if n==0:
                lent_front = 28*n
                x_data_front = np.zeros((28, lent_front, 1), dtype=np.float32)
                x_data_after = np.zeros((28, 72-lent_front, 1), dtype=np.float32)
            else:
                numb=np.random.randint(8,18)
                lent_front = lent_front+numb
                x_data_front = np.zeros((28, lent_front, 1), dtype=np.float32)
                x_data_after = np.zeros((28, 72-lent_front, 1), dtype=np.float32)
            canvas_add = np.concatenate([x_data_front, x_data[id]], 1)
            canvas_add = np.concatenate([canvas_add, x_data_after],  1)
            canvas += canvas_add
            labe[n] = y_data[id]
            
        datalabe = [str(x) for x in labe]
        data_labe[i] = ''.join(datalabe)
        data_numb[i]=canvas 
    print('end')
    return data_numb, data_labe

In [2]:
def get_label(y_lab):
    max_length = max(len(sublist[0]) for sublist in y_lab)
    lab_list = [[int(char) if len(char) == 1 else 11 for char in sublist[0]] + [10] * (max_length - len(sublist[0])) for sublist in y_lab]
    lab = np.array(lab_list)
    return lab

In [3]:
x_data_random , y_data_random = generate_random_number(100000,x_train, y_train)

end


In [4]:
y_data_ran = get_label(y_data_random)

In [5]:
from keras.utils import to_categorical
y_data_ran = to_categorical(y_data_ran, num_classes=11)

In [6]:
x_train_ran , y_train_ran,  y_train_random = x_data_random[:60000] , y_data_ran[:60000]  , y_data_random[:60000]
x_test_ran , y_test_ran,  y_test_random= x_data_random[60000:80000] , y_data_ran[60000:80000] , y_data_random[60000:80000]
x_verify_ran , y_verify_ran, y_verify_random = x_data_random[80000:] , y_data_ran[80000:] , y_data_random[80000:]

In [33]:
from PIL import Image
from IPython.display import display

def image_see(n):
    image_data = (np.squeeze(x_train_ran[n]) * 255).astype('uint8')
    image = Image.fromarray(image_data, 'L')
    title = f"Label:{y_train_random[n]}"
    print(title)
    display(image)
for id_n in np.random.randint(0, len(y_train_random),10):
    image_see(id_n)



Label:['809']


Label:['7']


Label:['08590']


Label:['76']


Label:['7493']


Label:['5']


Label:['2']


Label:['19009']


Label:['6']


Label:['72']


In [56]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Reshape, Dense, LSTM, Bidirectional
from tensorflow.keras.models import Model
from tensorflow import keras
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau 

input_shape = (28, 100, 1)

#CNN
input_img = Input(shape=input_shape, name="image", dtype="float32")
x = Conv2D(32, (3, 3), activation='tanh')(input_img)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='tanh')(x)
x = MaxPooling2D((2, 2))(x)
x = Reshape(target_shape=(5, 1472))(x)

#RNN
x = Bidirectional(LSTM(128, return_sequences=True, dropout=0.25))(x)
x = Bidirectional(LSTM(64, return_sequences=True, dropout=0.25))(x)

output = Dense(11, activation="softmax")(x) 

model = Model(inputs=input_img, outputs=output)

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01),
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image (InputLayer)           [(None, 28, 100, 1)]      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 26, 98, 32)        320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 49, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 47, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 23, 64)         0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 5, 1472)           0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 5, 256)           

In [8]:
reduce_lr = ReduceLROnPlateau(patience =3, factor = 0.01, weightdecay = 0.05, verbose = 1) 
early_stopping = keras.callbacks.EarlyStopping(monitor="val_loss", 
                                               patience=10, restore_best_weights=True)
#train model
history = model.fit(x_train_ran, y_train_ran, epochs=50, verbose=1, batch_size=200, 
          validation_data=(x_verify_ran, y_verify_ran), callbacks=[early_stopping, reduce_lr ])

Epoch 1/50
300/300 [==============================] - 110s 367ms/step - loss: 1.0726 - accuracy: 0.6261 - val_loss: 0.6162 - val_accuracy: 0.7937
Epoch 2/50
300/300 [==============================] - 98s 326ms/step - loss: 0.5633 - accuracy: 0.8123 - val_loss: 0.4323 - val_accuracy: 0.8590
Epoch 3/50
300/300 [==============================] - 90s 299ms/step - loss: 0.4550 - accuracy: 0.8507 - val_loss: 0.3790 - val_accuracy: 0.8784
Epoch 4/50
300/300 [==============================] - 89s 298ms/step - loss: 0.4099 - accuracy: 0.8659 - val_loss: 0.3331 - val_accuracy: 0.8923
Epoch 5/50
300/300 [==============================] - 91s 304ms/step - loss: 0.3824 - accuracy: 0.8748 - val_loss: 0.3231 - val_accuracy: 0.8956
Epoch 6/50
300/300 [==============================] - 89s 298ms/step - loss: 0.3644 - accuracy: 0.8812 - val_loss: 0.3241 - val_accuracy: 0.8963
Epoch 7/50
300/300 [==============================] - 90s 299ms/step - loss: 0.3520 - accuracy: 0.8854 - val_loss: 0.3022 - val_a

In [9]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2, subplot_titles=("Model accuracy", "Model loss"))

# Add the first subplot for accuracy
fig.add_trace(go.Scatter(x=list(range(1, len(history.history['accuracy']) + 1)),
                         y=history.history['accuracy'], mode='lines',
                         name='Train', line=dict(color='blue')), row=1, col=1)
fig.add_trace(go.Scatter(x=list(range(1, len(history.history['val_accuracy']) + 1)),
                         y=history.history['val_accuracy'], mode='lines',
                         name='Validation', line=dict(color='orange')), row=1, col=1)

# Add the second subplot for loss
fig.add_trace(go.Scatter(x=list(range(1, len(history.history['loss']) + 1)),
                         y=history.history['loss'], mode='lines',
                         name='Train', line=dict(color='blue'), showlegend=False), row=1, col=2)
fig.add_trace(go.Scatter(x=list(range(1, len(history.history['val_loss']) + 1)),
                         y=history.history['val_loss'], mode='lines',
                         name='Validation', line=dict(color='orange'), showlegend=False), row=1, col=2)

fig.update_layout(title="Model Training History",
                  xaxis_title="Epoch",
                  yaxis_title="Value",
                  legend=dict(x=0.85, y=1))

fig.show()

In [10]:
# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(x_test_ran, y_test_ran)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

625/625 [==============================] - 10s 16ms/step - loss: 0.2359 - accuracy: 0.9247
Test Accuracy: 92.47%


In [36]:
y_pred = model.predict(x_test_ran) 
y_pred = np.argmax(y_pred, axis=2)
y_ture = np.argmax(y_test_ran, axis=2)

In [53]:
str_arr = y_pred.astype(str)
str_arr = np.where(str_arr == '10', '', str_arr)
y_test_pred  = [''.join(row) for row in str_arr]
y_test_pred  = np.array(y_test_pred)

In [55]:
for id_n in np.random.randint(0, len(y_test_random),10):
    image_data = (np.squeeze(x_test_ran[id_n]) * 255).astype('uint8')
    image = Image.fromarray(image_data, 'L')
    title = f"Label:{y_test_random[id_n]}; Predicted:{y_test_pred[id_n]}"
    print(title)
    display(image)

Label:['40071']; Predicted:40171


Label:['1']; Predicted:1


Label:['43']; Predicted:43


Label:['616']; Predicted:616


Label:['9']; Predicted:9


Label:['84']; Predicted:84


Label:['1']; Predicted:1


Label:['54745']; Predicted:54745


Label:['0']; Predicted:0


Label:['84']; Predicted:84


In [52]:
from PIL import Image, ImageDraw, ImageFont
import numpy as np

id = list(range(len(x_test)))

np.random.shuffle(id)
id_exam=id[:100]
print(len(id_exam))
for h in id_exam:
    image_data = (np.squeeze(x_test_ran[h]) * 255).astype('uint8')
    image = Image.fromarray(image_data, 'L')
    draw = ImageDraw.Draw(image)
    font = ImageFont.load_default() 
    title=f"Lab_{y_test_random[h]}-Pre_{y_test_pred[h]}"
    name=title.replace(" ", "")
    name=name.replace("[", "")
    name=name.replace("]", "")
    file_name = "E:/pythonfile/images-cate/"+name+".png"
    image.save(file_name)

print("Images saved successfully.")

100
Images saved successfully.
